# Setting Up

In [3]:
!pip install datasets
!pip install transformers

In [1]:
#import libraries
#from datasets import load_dataset
import pandas as pd
import numpy as np
import torch
import librosa, librosa.display
from IPython.display import Audio
import matplotlib.pyplot as plt
import soundfile as sf
from pydub import AudioSegment
from torch.utils.data import Dataset, DataLoader, TensorDataset
#from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor

# Importing Data

In [2]:
train_df = pd.read_csv('/kaggle/input/common-voice/cv-valid-train.csv')
val_df = pd.read_csv('/kaggle/input/common-voice/cv-valid-dev.csv')

In [3]:
train_df.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-train/sample-000000.mp3,learn to recognize omens and follow them the o...,1,0,NaN,NaN,NaN,NaN
1,cv-valid-train/sample-000001.mp3,everything in the universe evolved he said,1,0,NaN,NaN,NaN,NaN
2,cv-valid-train/sample-000002.mp3,you came so that you could learn about your dr...,1,0,NaN,NaN,NaN,NaN
3,cv-valid-train/sample-000003.mp3,so now i fear nothing because it was those ome...,1,0,NaN,NaN,NaN,NaN
4,cv-valid-train/sample-000004.mp3,if you start your emails with greetings let me...,3,2,NaN,NaN,NaN,NaN


In [4]:
train_df.shape

(195776, 8)

In [5]:
val_df

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they se...,2,0,NaN,NaN,NaN,NaN
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage ent...,2,0,NaN,NaN,NaN,NaN
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be des...,3,0,NaN,NaN,NaN,NaN
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could he...,1,0,fourties,female,england,NaN
...,...,...,...,...,...,...,...,...
4071,cv-valid-dev/sample-004071.mp3,but they could never have taught him arabic,2,1,NaN,NaN,NaN,NaN
4072,cv-valid-dev/sample-004072.mp3,he decided to concentrate on more practical ma...,1,0,NaN,NaN,NaN,NaN
4073,cv-valid-dev/sample-004073.mp3,that's what i'm not supposed to say,2,0,thirties,male,us,NaN
4074,cv-valid-dev/sample-004074.mp3,just handling them made him feel better,3,0,NaN,NaN,NaN,NaN


In [6]:
val_df.shape

(4076, 8)

# Data Preprocessing

In [7]:
def show_cat(df):
    print('teens', df.age.loc[df.age == 'teens'].count())
    print('twenties', df.age.loc[df.age == 'twenties'].count())
    print('thirties', df.age.loc[df.age == 'thirties'].count())
    print('fourties', df.age.loc[df.age == 'fourties'].count())
    print('fifties', df.age.loc[df.age == 'fifties'].count())
    print('sixties', df.age.loc[df.age == 'sixties'].count())
    print('seventies', df.age.loc[df.age == 'seventies'].count())
    print('eighties', df.age.loc[df.age == 'eighties'].count())
    return

In [8]:
# selecting the required fields
train_age_df = train_df.loc[:,['filename','age']]

In [9]:
train_age_df.head()

,filename,age
0,cv-valid-train/sample-000000.mp3,NaN
1,cv-valid-train/sample-000001.mp3,NaN
2,cv-valid-train/sample-000002.mp3,NaN
3,cv-valid-train/sample-000003.mp3,NaN
4,cv-valid-train/sample-000004.mp3,NaN


In [10]:
train_age_df.shape

(195776, 2)

In [11]:
train_age_df.fillna(0.0, inplace=True)

In [12]:
train_age_df

,filename,age
0,cv-valid-train/sample-000000.mp3,0.0
1,cv-valid-train/sample-000001.mp3,0.0
2,cv-valid-train/sample-000002.mp3,0.0
3,cv-valid-train/sample-000003.mp3,0.0
4,cv-valid-train/sample-000004.mp3,0.0
...,...,...
195771,cv-valid-train/sample-195771.mp3,thirties
195772,cv-valid-train/sample-195772.mp3,0.0
195773,cv-valid-train/sample-195773.mp3,0.0
195774,cv-valid-train/sample-195774.mp3,twenties


In [13]:
train_age_df.columns

Index(['filename', 'age'], dtype='object')

In [14]:
train_age_df = train_age_df[train_age_df['age']!=0.0]

In [15]:
train_age_df.head()

,filename,age
5,cv-valid-train/sample-000005.mp3,twenties
8,cv-valid-train/sample-000008.mp3,seventies
13,cv-valid-train/sample-000013.mp3,thirties
14,cv-valid-train/sample-000014.mp3,sixties
19,cv-valid-train/sample-000019.mp3,fifties


In [16]:
train_age_df.shape

(73768, 2)

In [17]:
train_age_df['age'].unique()

array(['twenties', 'seventies', 'thirties', 'sixties', 'fifties',
       'fourties', 'teens', 'eighties'], dtype=object)

In [18]:
train_age_df.loc[(train_age_df['age']=='eighties'),'age'] ='seventies'

In [19]:
train_age_df['age'].unique()

array(['twenties', 'seventies', 'thirties', 'sixties', 'fifties',
       'fourties', 'teens'], dtype=object)

In [20]:
train_age_df.iloc[0,0][-17:-4]

'sample-000005'

In [21]:
show_cat(train_age_df)

teens 5441
twenties 23003
thirties 18303
fourties 11100
fifties 9466
sixties 4584
seventies 1871
eighties 0


In [22]:
# selecting 1800 samples from each category
age_groups = train_age_df['age'].unique()

In [23]:
final_df = pd.DataFrame(columns= ['filepath', 'age'])
for age_grp in age_groups:
    final_df = pd.concat([final_df, pd.DataFrame(train_age_df[train_age_df['age']==age_grp].sample(1000))], axis =0, ignore_index=True)
final_df.shape

(7000, 3)

In [68]:
# creating final_df_2 for fine tuning the model
final_df_2_ = train_age_df.merge(final_df, how='left', indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])
final_df_2_.shape

(66768, 3)

In [69]:
show_cat(final_df_2_)

teens 4441
twenties 22003
thirties 17303
fourties 10100
fifties 8466
sixties 3584
seventies 871
eighties 0


In [70]:
final_df_2 = pd.DataFrame(columns= ['filepath', 'age'])
for age_grp in age_groups:
    final_df_2 = pd.concat([final_df_2, pd.DataFrame(final_df_2_[final_df_2_['age']==age_grp].sample(870))], axis =0, ignore_index=True)
final_df_2.shape

(6090, 3)

## Segmenting into 3 second voice clips

In [24]:
audio = AudioSegment.from_file('/kaggle/input/common-voice/cv-valid-train/' +train_age_df.iloc[0,0], format="mp3")

In [25]:
audio

In [26]:
segments = [audio[i:i+3000] for i in range(0, len(audio), 3000)]

In [27]:
segments

In [28]:
segments[0].export("/kaggle/working/"+"test_segment_1.wav", format="wav")


<_io.BufferedRandom name='/kaggle/working/test_segment_1.wav'>

In [29]:
def length_fixing(sample):
    
    # each input is defined to have a 1s (1000ms) length
    segment_length = 3000
    
    temp_df = pd.DataFrame(columns= ['filepath', 'age'])
    
    audio_file_path =  '/kaggle/input/common-voice/cv-valid-train/' + sample['filename']
    file_name = sample['filename'][-17:-4]

    # loading audio using AudioSegment
    audio = AudioSegment.from_file(audio_file_path, format="mp3")
    
    # print(len(audio))
    if len(audio) < 3000:
        padding = AudioSegment.silent(duration=(segment_length - len(audio)))
        audio += padding
        # print(len(audio))
        audio.export("/kaggle/working/"+ file_name +"segment_1.wav", format="wav")
        new_record ={
                'filepath' : ["/kaggle/working/"+ file_name +"segment_1.wav"],
                'age' : [sample['age']]
        }
        return pd.concat([temp_df, pd.DataFrame(new_record)], ignore_index=True)    
    
    segments = [audio[i:i+segment_length] for i in range(0, len(audio), segment_length)]
    
    # padding the last segment to match the fixed length
    last_seg_len = len(segments[-1])
    if last_seg_len > 2500:
        padding = AudioSegment.silent(duration=(segment_length - last_seg_len))
        segments[-1] += padding
    else:
        segments = segments[:-1]
    # print(len(segments[-1]))
    for i,segment in enumerate(segments):
            segment.export("/kaggle/working/"+ file_name +f"segment_{i}.wav", format="wav")
            new_record ={
                'filepath' : ["/kaggle/working/"+ file_name +f"segment_{i}.wav"],
                'age' : [sample['age']]
            }
            temp_df = pd.concat([temp_df, pd.DataFrame(new_record)], ignore_index=True)    
    return temp_df

In [30]:
train_df_seg = pd.DataFrame(columns= ['filepath', 'age'])
    
for _, row in final_df.iterrows():
    temp_df = length_fixing(row)
    train_df_seg = pd.concat([train_df_seg, temp_df], ignore_index=True)  

In [71]:
train_df_seg_2 = pd.DataFrame(columns= ['filepath', 'age'])
    
for _, row in final_df_2.iterrows():
    temp_df = length_fixing(row)
    train_df_seg_2 = pd.concat([train_df_seg_2, temp_df], ignore_index=True)  

In [31]:
train_df_seg.head()

,filepath,age
0,/kaggle/working/sample-158177segment_0.wav,twenties
1,/kaggle/working/sample-006330segment_0.wav,twenties
2,/kaggle/working/sample-001930segment_0.wav,twenties
3,/kaggle/working/sample-117078segment_0.wav,twenties
4,/kaggle/working/sample-117078segment_1.wav,twenties


In [32]:
train_df_seg.shape

(8577, 2)

In [72]:
train_df_seg_2.shape

(7562, 2)

In [33]:
show_cat(train_df_seg)

teens 1115
twenties 1179
thirties 1141
fourties 1204
fifties 1209
sixties 1345
seventies 1384
eighties 0


In [73]:
show_cat(train_df_seg_2)

teens 996
twenties 983
thirties 1034
fourties 1049
fifties 1081
sixties 1130
seventies 1289
eighties 0


## Train Test Split

In [36]:
# train : val = 0.8:0.2

In [34]:
train_df = pd.DataFrame(columns= ['filepath', 'age'])
for age_grp in age_groups:
    train_df = pd.concat([train_df, pd.DataFrame(train_df_seg[train_df_seg['age']==age_grp].sample(880))], axis =0, ignore_index=True)
train_df.shape

(6160, 2)

In [74]:
train_df_2 = pd.DataFrame(columns= ['filepath', 'age'])
for age_grp in age_groups:
    train_df_2 = pd.concat([train_df_2, pd.DataFrame(train_df_seg_2[train_df_seg_2['age']==age_grp].sample(720))], axis =0, ignore_index=True)
train_df_2.shape

(5040, 2)

In [35]:
test_df = train_df_seg.merge(train_df, how='left', indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])
test_df.shape

(2417, 2)

In [75]:
test_df_2 = train_df_seg_2.merge(train_df_2, how='left', indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])
test_df_2.shape

(2522, 2)

In [79]:
show_cat(test_df_2)

teens 276
twenties 263
thirties 314
fourties 329
fifties 361
sixties 410
seventies 569
eighties 0


In [36]:
val_df = pd.DataFrame(columns= ['filepath', 'age'])
for age_grp in age_groups:
    val_df = pd.concat([val_df, pd.DataFrame(test_df[test_df['age']==age_grp].sample(220))], axis =0, ignore_index=True)
val_df.shape

(1540, 2)

In [77]:
val_df_2 = pd.DataFrame(columns= ['filepath', 'age'])
for age_grp in age_groups:
    val_df_2 = pd.concat([val_df_2, pd.DataFrame(test_df_2[test_df_2['age']==age_grp].sample(180))], axis =0, ignore_index=True)
val_df_2.shape

(1260, 2)

In [78]:
show_cat(val_df_2)

teens 180
twenties 180
thirties 180
fourties 180
fifties 180
sixties 180
seventies 180
eighties 0


## Mel Spectrogram

In [37]:
def extract_features(dataset):
    
    temp_feature_list = []
    temp_label_list = []
    
    # loop through th erows of dataframe
    
    for _, row in dataset.iterrows():
        
        file_path = row['filepath']
        label = row['age']
        #print('filepath:', file_path)
        #print('label:', label)
        # loading the audion file
        audio, sr = librosa.load(file_path, sr=44000)
        if len(audio) < sr*3:
            audio = librosa.util.pad_center(audio, size=sr*3)
        # print(len(audio))
        MFCCs = librosa.feature.mfcc(y=audio[:sr*3],sr=sr, n_fft=128,hop_length=512,n_mfcc=128)
        #print(MFCCs_.shape)
        # log spectro of the MFCCs
        MFCCs_ = librosa.amplitude_to_db(MFCCs)
        temp_feature_list.append(MFCCs_)
        temp_label_list.append(label)
        
    return np.array(temp_feature_list), np.array(temp_label_list)

In [38]:
X_train, Y_train = extract_features(train_df)

/opt/conda/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


In [80]:
X_train_2, Y_train_2 = extract_features(train_df_2)

/opt/conda/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


In [39]:
X_train.shape, Y_train.shape

((6160, 128, 258), (6160,))

In [81]:
X_train_2.shape, Y_train_2.shape

((5040, 128, 258), (5040,))

In [40]:
X_val, Y_val = extract_features(val_df)

In [82]:
X_val_2, Y_val_2 = extract_features(val_df_2)

In [41]:
X_val.shape, Y_val.shape

((1540, 128, 258), (1540,))

In [83]:
X_val_2.shape, Y_val_2.shape

((1260, 128, 258), (1260,))

## Creating Data Loaders

In [42]:
# converting the target class into one-hot-encoded vectors
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

Y_train_lb = lb.fit_transform(Y_train)
Y_val_lb = lb.fit_transform(Y_val)

In [84]:
Y_train_lb_2 = lb.fit_transform(Y_train_2)
Y_val_lb_2 = lb.fit_transform(Y_val_2)

In [43]:
Y_val_lb

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [44]:
Y_train_lb.shape, Y_val_lb.shape

((6160, 7), (1540, 7))

In [45]:
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train_lb, dtype=torch.float32)
X_test = torch.tensor(X_val, dtype=torch.float32)
Y_test = torch.tensor(Y_val_lb, dtype=torch.float32)

In [85]:
X_train_2 = torch.tensor(X_train_2, dtype=torch.float32)
Y_train_2 = torch.tensor(Y_train_lb_2, dtype=torch.float32)
X_test_2 = torch.tensor(X_val_2, dtype=torch.float32)
Y_test_2 = torch.tensor(Y_val_lb_2, dtype=torch.float32)

In [46]:
train_loader = DataLoader(TensorDataset(X_train, Y_train), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(X_test, Y_test), batch_size=64)

In [86]:
train_loader_2 = DataLoader(TensorDataset(X_train_2, Y_train_2), batch_size=64, shuffle=True)
val_loader_2 = DataLoader(TensorDataset(X_test_2, Y_test_2), batch_size=64)

# Models

In [59]:
# importing pre-trained versae/wav2vec2-base-finetuned-coscan-age_group model
age_model = Wav2Vec2ForSequenceClassification.from_pretrained("versae/wav2vec2-base-finetuned-coscan-age_group")

In [60]:
num_classes = 7
age_model.classifier = torch.nn.Linear(256, num_classes)

In [61]:
for name, param in age_model.named_parameters():
    if not name.startswith("classifier"):  
        param.requires_grad = False

## CNN Model

In [47]:
import torch
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1) # output - 128x258
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)# 64x129
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=128, kernel_size=3, stride=1, padding=1) # 64x129
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2,3), stride=2)# 32 x 64
        
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1) # 32 x 64
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)# 16x32
        
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, stride=1, padding=1) # 16x32
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)# 8x16
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 8 * 16, 4096)
        self.relu5 = nn.ReLU()
        self.fc2 = nn.Linear(4096, 1024)
        self.relu6 = nn.ReLU()
        self.fc3 = nn.Linear(1024, num_classes)

    def forward(self, x):
        #print(x.shape)
        #print(x)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.pool4(x)
        
        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        x = self.relu5(x)
        x = self.fc2(x)
        x = self.relu6(x)
        x = self.fc3(x)
        
        return x

# Training

In [48]:
optimizer = torch.optim.Adam(age_model.classifier.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

NameError: name 'age_model' is not defined

In [53]:
def train(model, iterator, optimizer, criterion):
    model.train()
    for batch in iterator:
        src, trg = batch
        src = src.unsqueeze(1)
        #print(src.shape)
        optimizer.zero_grad()
        output = model(src)
        # output = torch.mean(output, axis = 1)
        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()

In [54]:
def evaluate(model, iterator, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in iterator:
            src, trg = batch
            #src = src.view(64,-1)
            src = src.unsqueeze(1)
            output = model(src)
            #output = torch.mean(output, axis = 1)
            loss = criterion(output, trg)
            total_loss += loss.item()
    return total_loss / len(iterator)

In [55]:
from sklearn.metrics import accuracy_score, confusion_matrix
def calculate_accuracy(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        predicted_labels = []
        true_labels = []
        for data in data_loader:
            inputs, labels = data
            #inputs = inputs.view(64,-1)
            inputs = inputs.unsqueeze(1)
            output = model(inputs)
            #print(output.shape)
            #output = torch.mean(output, axis = 1)
            #print(output)
            max_index = (torch.argmax(output,dim =1).view(-1)).numpy()
            #print("max: ",len(max_index))
            true_label = (torch.argmax(labels,dim =1).view(-1)).numpy()
            #print("true:" ,len(true_label))
            predicted_labels.extend(max_index)
            true_labels.extend(true_label)
    #print(predicted_labels[:-30])
    #print(len(true_labels))
    accuracy = accuracy_score(true_labels, predicted_labels)
    print('Confusion_matrix: ',confusion_matrix(true_labels, predicted_labels))
    return accuracy

## Training the model for 5 epochs

In [1]:
N_EPOCHS = 5
for epoch in range(N_EPOCHS):
    train(age_model, train_loader, optimizer, criterion)
    val_loss = evaluate(age_model, val_loader, criterion)
    print(f'Epoch: {epoch+1:02}')
    #print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tValidation Loss: {val_loss:.3f}')

NameError: name 'train' is not defined

## Training the CNN Model

In [49]:
CNN_model = SimpleCNN(7)

# Print the model summary
print(CNN_model)

SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=(2, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu4): ReLU()
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=4096, bias=True)
  (relu5): ReLU()
  (fc2): Linear(in_features=4096, out_features=1024, bias=True)
  (relu6): ReLU()
  (fc3): Linear(in_features=1024, out_features=7, bias=True)
)


In [87]:
# defining the optimizers and loss functions
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(CNN_model.parameters(), lr=0.0001)

In [56]:
N_EPOCHS = 5
for epoch in range(N_EPOCHS):
    train(CNN_model, train_loader, optimizer, criterion)
    val_loss = evaluate(CNN_model, val_loader, criterion)
    print(f'Epoch: {epoch+1:02}')
    #print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tValidation Loss: {val_loss:.3f}')

Epoch: 01
	Validation Loss: 1.902
Epoch: 02
	Validation Loss: 1.891
Epoch: 03
	Validation Loss: 1.880
Epoch: 04
	Validation Loss: 1.897
Epoch: 05
	Validation Loss: 1.892


In [57]:
# Calculate accuracy on the validation set
train_accuracy = calculate_accuracy(CNN_model, train_loader)

val_accuracy = calculate_accuracy(CNN_model, val_loader)

Confusion_matrix:  [[309  53 230 125  79  42  42]
 [155 142 233 145 121  50  34]
 [ 67  41 537 111  93  14  17]
 [118  22 218 378  85  32  27]
 [ 64  43 125  73 449  58  68]
 [144  59 204 113 162 157  41]
 [ 90  37 173 124 261  67 128]]
Confusion_matrix:  [[ 56  17  53  45  27  16   6]
 [ 33  25  68  34  33  19   8]
 [ 21   9 112  33  25   9  11]
 [ 32   7  57  76  29   7  12]
 [ 21  11  42  18  87  24  17]
 [ 27  24  36  35  56  25  17]
 [ 27  19  46  33  56  10  29]]


In [58]:
train_accuracy, val_accuracy

(0.3409090909090909, 0.2662337662337662)

In [59]:
CNN_model_temp = CNN_model

In [60]:
# training for 5 more epochs
N_EPOCHS = 5
for epoch in range(N_EPOCHS):
    train(CNN_model_temp, train_loader, optimizer, criterion)
    val_loss = evaluate(CNN_model_temp, val_loader, criterion)
    print(f'Epoch: {epoch+1:02}')
    #print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tValidation Loss: {val_loss:.3f}')

Epoch: 01
	Validation Loss: 1.912
Epoch: 02
	Validation Loss: 1.877
Epoch: 03
	Validation Loss: 1.909
Epoch: 04
	Validation Loss: 1.962
Epoch: 05
	Validation Loss: 2.314


In [61]:
# Calculate accuracy on the validation set
train_accuracy = calculate_accuracy(CNN_model_temp, train_loader)

val_accuracy = calculate_accuracy(CNN_model_temp, val_loader)

Confusion_matrix:  [[518 162  46  32  41  61  20]
 [ 34 657  50  20  42  52  25]
 [ 14  75 717  22  23  13  16]
 [ 48  73  60 599  35  52  13]
 [ 26  58  32  14 620  70  60]
 [ 35  79  32  23  75 599  37]
 [ 28  94  45  38  88  87 500]]
Confusion_matrix:  [[38 48 25 26 22 41 20]
 [22 86 28 20 18 31 15]
 [21 36 84 20 14 27 18]
 [20 27 40 74 20 25 14]
 [14 34 14 16 68 36 38]
 [19 50 17 16 34 63 21]
 [27 43 17 23 32 36 42]]


In [62]:
train_accuracy, val_accuracy

(0.6834415584415584, 0.29545454545454547)

In [65]:
# CNN_model_temp has overfitted 
# hence fine tune CNN_model on a different dataset

In [89]:
# training for 5 more epochs
N_EPOCHS = 5
for epoch in range(N_EPOCHS):
    train(CNN_model, train_loader_2, optimizer, criterion)
    val_loss = evaluate(CNN_model, val_loader_2, criterion)
    print(f'Epoch: {epoch+1:02}')
    #print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tValidation Loss: {val_loss:.3f}')

Epoch: 01
	Validation Loss: 1.710
Epoch: 02
	Validation Loss: 1.714
Epoch: 03
	Validation Loss: 1.788
Epoch: 04
	Validation Loss: 1.957
Epoch: 05
	Validation Loss: 2.318


In [90]:
# Calculate accuracy on the validation set
train_accuracy = calculate_accuracy(CNN_model, train_loader_2)

val_accuracy = calculate_accuracy(CNN_model, val_loader_2)

Confusion_matrix:  [[686   4   5   6   3  13   3]
 [ 68 515   9  23  30  50  25]
 [ 27   5 623  21   9  20  15]
 [ 15   3   4 685   3   4   6]
 [ 39   4   5  17 625  14  16]
 [ 39   7   4  10  12 631  17]
 [ 40   7   7  32  24  22 588]]
Confusion_matrix:  [[78  8 18 22 14 23 17]
 [46 25 11 29 23 26 20]
 [23  3 76 24 11 26 17]
 [29 13 12 80 11 23 12]
 [26  7 15 13 61 28 30]
 [32 12  9 26 18 66 17]
 [31 13 13 14 37 41 31]]


In [91]:
train_accuracy, val_accuracy

(0.8636904761904762, 0.33095238095238094)

In [ ]:
N_EPOCHS = 5
for epoch in range(N_EPOCHS):
    train(CNN_model, train_loader, optimizer, criterion)
    val_loss = evaluate(CNN_model, val_loader, criterion)
    print(f'Epoch: {epoch+1:02}')
    #print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tValidation Loss: {val_loss:.3f}')

In [ ]:
# Calculate accuracy on the validation set
train_accuracy = calculate_accuracy(CNN_model, train_loader_2)

val_accuracy = calculate_accuracy(CNN_model, val_loader_2)

In [ ]:
train_accuracy, val_accuracy

In [63]:
# saving the model
import pickle
# saving models in both .pkl and .pth formats

In [64]:
torch.save(CNN_model, '/kaggle/working/Age_CNN.pth')

with open('/kaggle/working/Age_CNN.pkl', 'wb') as file:
    pickle.dump(CNN_model, file)
    
torch.save(CNN_model, '/kaggle/working/Age_CNN.h5')

# Downloading the Models

In [66]:
from IPython.display import FileLink

In [67]:
FileLink(r'Age_CNN.pth')

/kaggle/working/Age_CNN.pth

In [68]:
FileLink(r'Age_CNN.pkl')

/kaggle/working/Age_CNN.pkl

In [69]:
FileLink(r'Age_CNN.h5')

/kaggle/working/Age_CNN.h5